In [1]:
import tensorflow as tf
import numpy as np

C:\Users\home\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
slim = tf.contrib.slim

In [3]:
x = tf.random_normal(shape=[16,28,28,3])

In [4]:
g = tf.random_normal(shape=[16,28,28,3])

In [5]:
def count_parameters(network_variables, name):
    """
    This method counts the tota
    l number of unique parameters for a list of variable objects
    :param network_variables: A list of tf network variable objects
    :param name: Name of the network
    """
    total_parameters = 0
    for variable in network_variables:
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        variable_parametes = 1
        for dim in shape:
            variable_parametes *= dim.value

        total_parameters += variable_parametes
    print(name, total_parameters)

In [7]:
class Discriminator:
    def __init__(self, batch_size, layer_sizes, inner_layers, use_wide_connections=False, name="d"):
        """
        Initialize a discriminator network.
        :param batch_size: Batch size for discriminator.
        :param layer_sizes: A list with the feature maps for each MultiLayer.
        :param inner_layers: An integer indicating the number of inner layers.
        """
        self.reuse = False
        self.batch_size = batch_size
        self.layer_sizes = layer_sizes
        self.inner_layers = inner_layers
        self.conv_layer_num = 0
        self.use_wide_connections = use_wide_connections
        self.build = True
        self.name = name
        self.current_layers = []
        
    def upscale(self, x, scale):
        """
            Upscales an image using nearest neighbour
            :param x: Input image
            :param h_size: Image height size
            :param w_size: Image width size
            :return: Upscaled image
        """
        [b, h, w, c] = [int(dim) for dim in x.get_shape()]

        return tf.image.resize_nearest_neighbor(x, (h * scale, w * scale))
        
    def encoder_block(self, x1, num): #last output, previous_output 
    #with tf.variable_scope(np.str(num)+'encoder_block'):
        with slim.arg_scope([slim.conv2d, slim.conv2d_transpose],
                            num_outputs = 64, padding = 'SAME',
                            kernel_size = [3,3], stride = (1,1),
                            activation_fn = tf.nn.leaky_relu,
                            normalizer_fn=slim.batch_norm, normalizer_params= self.batch_norm_params):

            conv1_1 = slim.conv2d(x1)
            self.current_layers.append(conv1_1)
            output1_1 = tf.concat([conv1_1, x1], axis=3)
            print(output1_1)

            conv1_2 = slim.conv2d(output1_1)
            self.current_layers.append(conv1_2)
            output1_2 = tf.concat([conv1_2, output1_1], axis=3)
            print(output1_2)

            conv1_3 = slim.conv2d(output1_2)
            self.current_layers.append(conv1_3)
            output1_3 = tf.concat([conv1_3, output1_2], axis=3)
            print(output1_3)

            conv1_4 = slim.conv2d(output1_3)
            self.current_layers.append(conv1_4)
            output1_4 = tf.concat([conv1_4, output1_3], axis=3)
            print(output1_4)

            conv1_5 = slim.conv2d(output1_4, stride=(2,2))
            self.current_layers.append(conv1_5)
            output = slim.dropout(conv1_5, keep_prob=0.5)
            self.current_layers.append(output)
            print(output)

            if num == 3:
                pass
            else :
                input_projection = slim.conv2d(conv1_4, num_outputs=conv1_4.get_shape()[3], stride=(2,2),
                                               activation_fn= None, normalizer_fn= None)
                output = tf.concat([output, input_projection], axis=3)
            print(output)

        return output
    
    def __call__(self, conditional_input, generated_input, training=False, dropout_rate=0.0):
        """
        :param conditional_input: A batch of conditional inputs (x_i) of size [batch_size, height, width, channel]
        :param generated_input: A batch of generated inputs (x_g) of size [batch_size, height, width, channel]
        :param training: Placeholder for training or a boolean indicating training or validation
        :param dropout_rate: A float placeholder for dropout rate or a float indicating the dropout rate
        :param name: Network name
        :return:
        """
        self.batch_norm_params = {'decay' : 0.99, 'scale' : True, 'center' : True,
                                 'is_training' : training, 'renorm' : True}
        conditional_input = tf.convert_to_tensor(conditional_input)
        generated_input = tf.convert_to_tensor(generated_input)
        
        
        #with tf.variable_scope(self.name, reuse=self.reuse):
        concat_images = tf.concat([conditional_input, generated_input], axis=3)
        outputs = concat_images
        self.current_layers.append(outputs)
               
        with slim.arg_scope([slim.conv2d, slim.conv2d_transpose],
                        num_outputs = 64, padding = 'SAME',
                        kernel_size = [3,3], stride = (1,1),
                        activation_fn = tf.nn.leaky_relu,
                        normalizer_fn=slim.batch_norm, normalizer_params= self.batch_norm_params):
            
            #with tf.variable_scope('first_conv'): 
            conv = slim.conv2d(outputs, stride=(2,2))
            self.current_layers.append(conv)

            input_projection = slim.conv2d(outputs, num_outputs=outputs.get_shape()[3], kernel_size = [3,3], stride=(2,2),
                                   activation_fn= None, normalizer_fn= None)
            conv1 = tf.concat([conv, input_projection], axis=3)

            en1 = self.encoder_block(conv1, 1)
            en2 = self.encoder_block(en1, 2)
            en3 = self.encoder_block(en2, 3)

        #with tf.variable_scope('decoder_block'):
            feature_level_flatten = tf.reduce_mean(en3, axis=[1, 2])
            print('else_feature_level_flatten : ', feature_level_flatten)
            location_level_flatten = tf.layers.flatten(en3)
            print('else_location_level_flatten : ', location_level_flatten)

            feature_level_dense = tf.layers.dense(feature_level_flatten, units=1024, activation=tf.nn.leaky_relu)
            combo_level_flatten = tf.concat([feature_level_dense, location_level_flatten], axis=1)

        #with tf.variable_scope('discriminator_out_block'):
            outputs = tf.layers.dense(combo_level_flatten, 1, name='outputs')

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        
        if self.build:
            print("discr layers", self.conv_layer_num)
            count_parameters(self.variables, name="discriminator_parameter_num")
        self.build = False
        
        return outputs, self.current_layers
                
       

In [8]:
a = Discriminator(16, [64,64,64,64], [5,5,5,5])

In [9]:
a(x, g)

Tensor("concat_1:0", shape=(16, 14, 14, 134), dtype=float32)
Tensor("concat_2:0", shape=(16, 14, 14, 198), dtype=float32)
Tensor("concat_3:0", shape=(16, 14, 14, 262), dtype=float32)
Tensor("concat_4:0", shape=(16, 14, 14, 326), dtype=float32)
Tensor("Dropout/dropout_1/mul:0", shape=(16, 7, 7, 64), dtype=float32)
Tensor("concat_5:0", shape=(16, 7, 7, 128), dtype=float32)
Tensor("concat_6:0", shape=(16, 7, 7, 192), dtype=float32)
Tensor("concat_7:0", shape=(16, 7, 7, 256), dtype=float32)
Tensor("concat_8:0", shape=(16, 7, 7, 320), dtype=float32)
Tensor("concat_9:0", shape=(16, 7, 7, 384), dtype=float32)
Tensor("Dropout_1/dropout/mul:0", shape=(16, 4, 4, 64), dtype=float32)
Tensor("concat_10:0", shape=(16, 4, 4, 128), dtype=float32)
Tensor("concat_11:0", shape=(16, 4, 4, 192), dtype=float32)
Tensor("concat_12:0", shape=(16, 4, 4, 256), dtype=float32)
Tensor("concat_13:0", shape=(16, 4, 4, 320), dtype=float32)
Tensor("concat_14:0", shape=(16, 4, 4, 384), dtype=float32)
Tensor("Dropout_2/d

(<tf.Tensor 'outputs/BiasAdd:0' shape=(16, 1) dtype=float32>,
 [<tf.Tensor 'd/concat:0' shape=(16, 28, 28, 6) dtype=float32>,
  <tf.Tensor 'Conv/LeakyRelu:0' shape=(16, 14, 14, 64) dtype=float32>,
  <tf.Tensor 'Conv_2/LeakyRelu:0' shape=(16, 14, 14, 64) dtype=float32>,
  <tf.Tensor 'Conv_3/LeakyRelu:0' shape=(16, 14, 14, 64) dtype=float32>,
  <tf.Tensor 'Conv_4/LeakyRelu:0' shape=(16, 14, 14, 64) dtype=float32>,
  <tf.Tensor 'Conv_5/LeakyRelu:0' shape=(16, 14, 14, 64) dtype=float32>,
  <tf.Tensor 'Conv_6/LeakyRelu:0' shape=(16, 7, 7, 64) dtype=float32>,
  <tf.Tensor 'Dropout/dropout_1/mul:0' shape=(16, 7, 7, 64) dtype=float32>,
  <tf.Tensor 'Conv_8/LeakyRelu:0' shape=(16, 7, 7, 64) dtype=float32>,
  <tf.Tensor 'Conv_9/LeakyRelu:0' shape=(16, 7, 7, 64) dtype=float32>,
  <tf.Tensor 'Conv_10/LeakyRelu:0' shape=(16, 7, 7, 64) dtype=float32>,
  <tf.Tensor 'Conv_11/LeakyRelu:0' shape=(16, 7, 7, 64) dtype=float32>,
  <tf.Tensor 'Conv_12/LeakyRelu:0' shape=(16, 4, 4, 64) dtype=float32>,
  <tf.

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    a = Discriminator(16, [64,64,64,64], [5,5,5,5])
    print(sess.run(a(x,g)))

Tensor("concat_17:0", shape=(16, 14, 14, 134), dtype=float32)
Tensor("concat_18:0", shape=(16, 14, 14, 198), dtype=float32)
Tensor("concat_19:0", shape=(16, 14, 14, 262), dtype=float32)
Tensor("concat_20:0", shape=(16, 14, 14, 326), dtype=float32)
Tensor("Dropout_3/dropout/mul:0", shape=(16, 7, 7, 64), dtype=float32)
Tensor("concat_21:0", shape=(16, 7, 7, 128), dtype=float32)
Tensor("concat_22:0", shape=(16, 7, 7, 192), dtype=float32)
Tensor("concat_23:0", shape=(16, 7, 7, 256), dtype=float32)
Tensor("concat_24:0", shape=(16, 7, 7, 320), dtype=float32)
Tensor("concat_25:0", shape=(16, 7, 7, 384), dtype=float32)
Tensor("Dropout_4/dropout/mul:0", shape=(16, 4, 4, 64), dtype=float32)
Tensor("concat_26:0", shape=(16, 4, 4, 128), dtype=float32)
Tensor("concat_27:0", shape=(16, 4, 4, 192), dtype=float32)
Tensor("concat_28:0", shape=(16, 4, 4, 256), dtype=float32)
Tensor("concat_29:0", shape=(16, 4, 4, 320), dtype=float32)
Tensor("concat_30:0", shape=(16, 4, 4, 384), dtype=float32)
Tensor("Dr

ValueError: Variable outputs/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\home\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1717, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Users\home\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3155, in create_op
    op_def=op_def)
  File "C:\Users\home\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 454, in new_func
    return func(*args, **kwargs)


In [ ]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)